# LoFTR demo resultados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/zju3dv/LoFTR.git
%cd LoFTR
#!pip install -r requirements.txt  # contiene torch, torchvision, etc.

In [ ]:
!pip install numpy==1.23.5 --force-reinstall

In [ ]:
!pip install opencv-python==4.5.5.64
!pip install albumentations==1.3.1
!pip install ray einops==0.3.0 loguru yacs tqdm matplotlib h5py joblib
!pip install kornia==0.6.7
!pip install torchmetrics==0.11.4 pytorch-lightning==1.9.4

In [ ]:
import os, cv2, torch, numpy as np, pandas as pd
from tqdm import tqdm
from scipy.spatial.distance import cdist
from scipy.stats import entropy
from loftr import LoFTR, default_cfg

# Preparar modelo
device = torch.device("cpu")
cfg = default_cfg
matcher = LoFTR(cfg=cfg)
matcher.to(device).eval()

def extract_loftr(img1, img2):
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)[None][None]
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)[None][None]
    t1 = torch.from_numpy(img1_gray / 255.).float().to(device)
    t2 = torch.from_numpy(img2_gray / 255.).float().to(device)
    with torch.no_grad():
        input_dict = {"image0": t1, "image1": t2}
        matched = matcher(input_dict)
    mkpts0 = matched["keypoints0"].cpu().numpy()
    mkpts1 = matched["keypoints1"].cpu().numpy()
    descriptors0 = matched["descriptors0"].cpu().numpy()
    descriptors1 = matched["descriptors1"].cpu().numpy()
    return mkpts0, descriptors0, mkpts1, descriptors1

def spatial_entropy(kps, shape, grid_size=4):
    H, W = shape[:2]
    heatmap = np.zeros((grid_size, grid_size))
    for x,y in kps:
        col = min(int(x/W*grid_size), grid_size-1)
        row = min(int(y/H*grid_size), grid_size-1)
        heatmap[row, col] += 1
    p = heatmap.flatten(); p /= (p.sum()+1e-8)
    return entropy(p)

def match_and_metrics(kp1, desc1, kp2, desc2):
    d = cdist(desc1, desc2)
    idx = np.argmin(d, axis=1)
    matched_kp1 = kp1
    distances = d[np.arange(len(idx)), idx]
    num_matches = len(idx)
    efficiency = num_matches / (len(desc1)+1e-8)
    avg_distance = float(distances.mean())
    spatial_ent = spatial_entropy(matched_kp1, img1.shape)
    return idx, distances, num_matches, efficiency, avg_distance, spatial_ent

def draw_matches(img1, kp1, img2, kp2, idx, save_path):
    h1,w1 = img1.shape[:2]
    out = np.zeros((max(img1.shape[0],img2.shape[0]), w1+img2.shape[1], 3),dtype=np.uint8)
    out[:h1,:w1]=img1; out[:img2.shape[0],w1:]=img2
    for i,j in enumerate(idx):
        pt1 = tuple(np.round(kp1[i]).astype(int))
        pt2 = tuple(np.round(kp2[j]).astype(int)+np.array([w1,0]))
        cv2.line(out,pt1,pt2,(0,255,0),1)
    cv2.imwrite(save_path,out)


In [ ]:
# Configuración
BASE = '/content/drive/MyDrive/pruebas-pfc-2025/aachen_dataset/day'
OUT = '/content/loftr_day_results'
os.makedirs(OUT, exist_ok=True)

results = []

# Evaluar
for pair in tqdm(sorted(os.listdir(BASE))):
    p = os.path.join(BASE, pair)
    i1,i2 = os.path.join(p,'img1.png'), os.path.join(p,'img2.png')
    if os.path.exists(i1) and os.path.exists(i2):
        img1 = cv2.imread(i1); img2 = cv2.imread(i2)
        kp1, desc1, kp2, desc2 = extract_loftr(img1, img2)
        idx, distances, nm, eff, ad, se = match_and_metrics(kp1, desc1, kp2, desc2)
        draw_matches(img1, kp1, img2, kp2, idx, os.path.join(OUT, f"{pair}_m.png"))
        results.append({'pair': pair, 'num_matches': nm,
                         'efficiency': round(eff,4),
                         'avg_distance': round(ad,4),
                         'spatial_entropy': round(se,4)})

# Guardar CSV
df = pd.DataFrame(results)
df.to_csv(os.path.join(OUT, 'loftr_day_results.csv'),index=False)